# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1VERPjEZcC1XSs4-02aM-DbkNr_yaJVbFjLJxaYQswqA/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_**All code below should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach.**_

## Requirements

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project._

* Code clarity: make sure the code conforms to:
    * [ ] [PEP 8](https://peps.python.org/pep-0008/) - You might find [this resource](https://realpython.com/python-pep8/) helpful as well as [this](https://github.com/dnanhkhoa/nb_black) or [this](https://jupyterlab-code-formatter.readthedocs.io/en/latest/) tool
    * [ ] [PEP 257](https://peps.python.org/pep-0257/)
    * [ ] Break each task down into logical functions
* The following files are submitted for the project (see the project's GDoc for more details):
    * [ ] `README.md`
    * [ ] `requirements.txt`
    * [ ] `.gitignore`
    * [ ] `schema.sql`
    * [ ] 6 query files (using the `.sql` extension), appropriately named for the purpose of the query
    * [x] Jupyter Notebook containing the project (this file!)
* [x] You can edit this cell and add a `x` inside the `[ ]` like this task to denote a completed task

## Project Setup

In [1]:
# all import statements needed for the project, for example:

import math
from math import radians, cos, sin, asin, sqrt
import bs4
import matplotlib.pyplot as plt
import pandas as pd
import requests
import sqlalchemy as db
from sqlalchemy import text
import re
import geopandas as gpd
from os.path import exists
#!pip install pyarrow
#!pip install fastparquet

In [2]:
# any general notebook setup, like log formatting

In [3]:
# any constants you might need, for example:

TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
# add other constants to refer to any local data, e.g. uber & weather
UBER_CSV = "uber_rides_sample.csv"

NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

## Part 1: Data Preprocessing

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Define a function that calculates the distance between two coordinates in kilometers that **only uses the `math` module** from the standard library.
* [ ] Taxi data:
    * [ ] Use the `re` module, and the packages `requests`, BeautifulSoup (`bs4`), and (optionally) `pandas` to programmatically download the required CSV files & load into memory.
    * You may need to do this one file at a time - download, clean, sample. You can cache the sampling by saving it as a CSV file (and thereby freeing up memory on your computer) before moving onto the next file. 
* [ ] Weather & Uber data:
    * [ ] Download the data manually in the link provided in the project doc.
* [ ] All data:
    * [ ] Load the data using `pandas`
    * [ ] Clean the data, including:
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * (Taxi & Uber data) Remove trips that start and/or end outside the designated [coordinate box](http://bboxfinder.com/#40.560445,-74.242330,40.908524,-73.717047)
    * [ ] (Taxi data) Sample the data so that you have roughly the same amount of data points over the given date range for both Taxi data and Uber data.
* [ ] Weather data:
    * [ ] Split into two `pandas` DataFrames: one for required hourly data, and one for the required daily daya.
    * [ ] You may find that the weather data you need later on does not exist at the frequency needed (daily vs hourly). You may calculate/generate samples from one to populate the other. Just document what you’re doing so we can follow along. 

### Calculating distance
_**TODO:** Write some prose that tells the reader what you're about to do here._

In [4]:
def calculate_distance(from_coord, to_coord):
    lat1=radians(from_coord[0])
    lat2=radians(to_coord[0])
    lon1=radians(from_coord[1])
    lon2=radians(to_coord[1])
    dis_lat=lat2-lat1
    dis_lon=lon2-lon1
    a = sin(dis_lat / 2)**2 + cos(lat1) * cos(lat2) * sin(dis_lon/ 2)**2
    c = 2 * asin(sqrt(a))
    return 6371*c

In [5]:
def add_distance_column(dataframe):
    distance_list = []
    for index, row in dataframe.iterrows():
        from_coord = []
        to_coord = []
        from_coord.append(row['pickup_latitude'])
        from_coord.append(row['pickup_longitude'])
        to_coord.append(row['dropoff_latitude'])
        to_coord.append(row['dropoff_longitude'])
        distance_list.append(calculate_distance(from_coord, to_coord))    
    dataframe['Distance'] = distance_list
    return dataframe

### Processing Taxi Data

**Getting Yellow Taxi URL**

In [6]:
def find_taxi_csv_urls():
    response = requests.get(TAXI_URL)
    former_result = response.content
    soup = bs4.BeautifulSoup(former_result, 'html.parser')
    
    result = []
    pattern = r"Yellow Taxi Trip Records"
    data = soup.find_all("a")
    for i in range(len(data)):
        matches = re.search(pattern, data[i].text)
        if type(matches) == re.Match:
            result.append(data[i]['href'])
    return result

In [7]:
find_taxi_csv_urls()

['https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-03.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-04.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-05.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-06.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-07.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-08.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-09.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-01.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-02.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-03.pa

**Define how to get shapefile to convert ID to coordinates**

In [8]:
def get_id_and_coordinate(shapefile):
    df = gpd.read_file(shapefile)
    df=df.to_crs(4326)
    #Find the center point
    df['Center_point'] = df['geometry'].centroid
    #Extract lat and lon from the centerpoint
    df["longitude"] = df.Center_point.map(lambda p: p.x)
    df["latitude"] = df.Center_point.map(lambda p: p.y)
    df = pd.concat([df["LocationID"], df["longitude"], df["latitude"]],axis = 1)
    return df

In [9]:
get_id_and_coordinate("taxi_zones.shp")
df_ID_coordinates_pickup = get_id_and_coordinate("taxi_zones.shp")
df_ID_coordinates_pickup.columns = ["PULocationID","pickup_longitude", "pickup_latitude"]

<ipython-input-8-ae9c73e78da5>:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df['Center_point'] = df['geometry'].centroid
<ipython-input-8-ae9c73e78da5>:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df['Center_point'] = df['geometry'].centroid


In [10]:
get_id_and_coordinate("taxi_zones.shp")
df_ID_coordinates_dropoff = get_id_and_coordinate("taxi_zones.shp")
df_ID_coordinates_dropoff.columns = ["DOLocationID","dropoff_longitude", "dropoff_latitude"]

<ipython-input-8-ae9c73e78da5>:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df['Center_point'] = df['geometry'].centroid
<ipython-input-8-ae9c73e78da5>:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df['Center_point'] = df['geometry'].centroid


**DownLoad Parquet**

In [11]:
def get_and_clean_month_taxi_data(url):
    #file_url = url
    pattern=r"[0-9]{4}-[0-9]{2}"
    match = re.search(pattern, url)
    fname=match.group()
    r = requests.get(url, stream = True)
    with open(f"python{fname}.parquet","wb") as parquet:
        for chunk in r.iter_content(chunk_size=100000):
            if chunk:
                parquet.write(chunk)

    name=f"python{fname}.parquet"
    df=pd.read_parquet(name,engine='auto')
    
    
   
    #df.columns = ["pickup_datetime", "dropoff_datetime", "trip_distance", "PULocationID", "DOLocationID", "tip_amount"]
    
    if "Trip_Pickup_DateTime" in df.columns: #For data in 2009
        df.rename(columns={"Trip_Pickup_DateTime": "pickup_datetime"}, inplace=True)
        df.rename(columns={"Trip_Dropoff_DateTime": "dropoff_datetime"}, inplace=True)
        df.rename(columns={"Trip_Distance": "trip_distance"}, inplace=True)
        df.rename(columns={"Start_Lon": "pickup_longitude"}, inplace=True)
        df.rename(columns={"Start_Lat": "pickup_latitude"}, inplace=True)
        df.rename(columns={"End_Lon": "dropoff_longitude"}, inplace=True)
        df.rename(columns={"End_Lat": "dropoff_latitude"}, inplace=True)
        df.rename(columns={"Tip_Amt": "tip_amount"}, inplace=True)
        
        df = pd.concat([
            df["pickup_datetime"], df["dropoff_datetime"], df["trip_distance"],
            df["pickup_longitude"],df["pickup_latitude"], 
            df["dropoff_longitude"],df["dropoff_latitude"],df["tip_amount"]],axis = 1)
    
    elif "pickup_datetime" in df.columns:  #For data in 2011
        df = pd.concat([
            df["pickup_datetime"], df["dropoff_datetime"], df["trip_distance"],
            df["pickup_longitude"],df["pickup_latitude"], 
            df["dropoff_longitude"],df["dropoff_latitude"],df["tip_amount"]],axis = 1)

    elif "tpep_pickup_datetime" in df.columns:  #For data from 2011 to 2022
        df.rename(columns={"tpep_pickup_datetime": "pickup_datetime"}, inplace=True)
        df.rename(columns={"tpep_dropoff_datetime": "dropoff_datetime"}, inplace=True)
        df = pd.concat([
            df["pickup_datetime"], df["dropoff_datetime"], df["trip_distance"], 
            df["PULocationID"], df["DOLocationID"],df["tip_amount"]],axis = 1)

        df = pd.merge(df, df_ID_coordinates_pickup, how = "left", on ="PULocationID")
        df = pd.merge(df, df_ID_coordinates_dropoff, how = "left", on ="DOLocationID")
        df.drop(["PULocationID","DOLocationID"],axis=1,inplace=True)
    
    return df


In [12]:
def get_and_clean_taxi_data():
    all_taxi_dataframes = []
    
    all_csv_urls = find_taxi_csv_urls()
    for csv_url in all_csv_urls:
        # maybe: first try to see if you've downloaded this exact file already and saved it before trying again

        # maybe: if the file hasn't been saved, save it so you can
        # avoid re-downloading it if you re-run the function
        pattern=r"[0-9]{4}-[0-9]{2}"
        match = re.search(pattern, csv_url)
        fname=match.group()
        
        if exists(f"Yellow_taxi{fname}.csv"):
            dataframe = pd.read_csv(f"Yellow_taxi{fname}.csv")
            
        else:
            dataframe = get_and_clean_month_taxi_data(csv_url)
            dataframe = dataframe.sample(n=2000, random_state=7) #generate samples that's roughly equal to Uber dataset
            
            add_distance_column(dataframe)
 
        for index, row in dataframe.iterrows():
            if row['pickup_latitude']<40.560445:
                dataframe = dataframe.drop(index)
            elif row['pickup_latitude']>40.908524:
                dataframe = dataframe.drop(index)
            elif row['dropoff_latitude']<40.560445:
                dataframe = dataframe.drop(index)
            elif row['dropoff_latitude']>40.908524:
                dataframe = dataframe.drop(index)
            elif row['pickup_longitude']<-74.242330:
                dataframe = dataframe.drop(index)
            elif row['pickup_longitude']>-73.717047:
                dataframe = dataframe.drop(index)
            elif row['dropoff_longitude']<-74.242330:
                dataframe = dataframe.drop(index) 
            elif row['dropoff_longitude']>-73.717047:
                dataframe = dataframe.drop(index)

        dataframe = dataframe.dropna(axis=0,how='any')

        dataframe.to_csv(f"Yellow_taxi{fname}.csv", index=False)
        #dataframe = pd.read_csv(f"Yellow_taxi{fname}.csv")

        all_taxi_dataframes.append(dataframe)
        
    # create one gigantic dataframe with data from every month needed
    taxi_data = pd.concat(all_taxi_dataframes)
    return taxi_data

In [13]:
#get_and_clean_taxi_data()

### Processing Uber Data

**Download Uber Data and remove trips out of the box** 

In [14]:
def load_and_clean_uber_data(csv_file):
    df = pd.read_csv(csv_file,low_memory=False)
    df.drop(["key","fare_amount","passenger_count","Unnamed: 0"],axis=1,inplace=True)
    for index, row in df.iterrows():
        if row['pickup_latitude']<40.560445:
            df = df.drop(index)
        elif row['pickup_latitude']>40.908524:
            df = df.drop(index)
        elif row['dropoff_latitude']<40.560445:
            df = df.drop(index)
        elif row['dropoff_latitude']>40.908524:
            df = df.drop(index)
        elif row['pickup_longitude']<-74.242330:
            df = df.drop(index)
        elif row['pickup_longitude']>-73.717047:
            df = df.drop(index)
        elif row['dropoff_longitude']<-74.242330:
            df = df.drop(index) 
        elif row['dropoff_longitude']>-73.717047:
            df = df.drop(index)
    return df

In [15]:
def get_uber_data():
    uber_dataframe = load_and_clean_uber_data(UBER_CSV)
    add_distance_column(uber_dataframe)
    return uber_dataframe

In [16]:
#get_uber_data()

### Processing Weather Data

**Get all the necessary columns in weather data**

**Fill in null values with 0 and mean**

In [17]:
def clean_month_weather_data_hourly(csv_file):
    original_df = pd.read_csv(csv_file ,low_memory=False)
    df = pd.concat([original_df["DATE"], original_df["HourlyPrecipitation"], original_df["HourlyWindSpeed"]],axis = 1)
    df["HourlyPrecipitation"] = df["HourlyPrecipitation"].replace('T', 0.00)
    df["HourlyPrecipitation"] = df["HourlyPrecipitation"].str.replace('s', '')
    df["HourlyPrecipitation"] = df["HourlyPrecipitation"].fillna(0.00)
    df["HourlyWindSpeed"] = df["HourlyWindSpeed"].fillna(0.00)
    df = df[df.DATE.str.contains(":51:00")]
    return df

In [18]:
def clean_month_weather_data_daily(csv_file):
    original_df = pd.read_csv(csv_file, low_memory=False)
    df = pd.concat([original_df["DATE"], original_df["DailyAverageWindSpeed"]],axis = 1)
    df = df[df.DATE.str.contains("23:59:00")]
    df["DailyAverageWindSpeed"] = df["DailyAverageWindSpeed"].fillna(df["DailyAverageWindSpeed"].astype(float).mean())
    df["DailyAverageWindSpeed"] = df["DailyAverageWindSpeed"].fillna(0.00)
    return df

In [19]:
def load_and_clean_weather_data():
    hourly_dataframes = []
    daily_dataframes = []
    
    weather_csv_files = ["2009_weather.csv", 
    "2010_weather.csv", "2011_weather.csv", 
    "2012_weather.csv","2013_weather.csv", 
    "2014_weather.csv", "2015_weather.csv",]
    
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    
    return hourly_data, daily_data

### Process All Data


_**Run all the functions above**_

In [20]:

uber_data = get_uber_data()
hourly_weather_data, daily_weather_data = load_and_clean_weather_data()
taxi_data = get_and_clean_taxi_data()

In [21]:
#hourly_weather_data

In [22]:
#daily_weather_data

In [23]:
#taxi_data

In [24]:
#uber_data

## Part 2: Storing Cleaned Data

_Write some prose that tells the reader what you're about to do here._

In [67]:
engine = db.create_engine(DATABASE_URL)
connection = engine.connect()

In [68]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS hourly_weather
(
    id INTEGER PRIMARY KEY,
    Date DATETIME,
        HourlyPrecipitation FLOAT,
HourlyWindSpeed FLOAT
);
"""

DAILY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS daily_weather
(
    id INTEGER PRIMARY KEY,
    Date DATETIME,
    DailyAverageWindSpeed FLOAT
);
"""

TAXI_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS taxi_trips
(
    id INTEGER PRIMARY KEY,
    pickup_datetime DATETIME,
    dropoff_datetime DATETIME,	
    trip_distance FLOAT,	
    tip_amount FLOAT,	
    pickup_longitude FLOAT,	
    pickup_latitude FLOAT,	
    dropoff_longitude FLOAT,	
    dropoff_latitude FLOAT,	
    Distance FLOAT
);

"""

UBER_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS uber_trips
(
    id INTEGER PRIMARY KEY,
    pickup_datetime  DATETIME,
    pickup_longitude FLOAT,
    pickup_latitude FLOAT,
    dropoff_longitude FLOAT,
    dropoff_latitude FLOAT,
    Distance FLOAT
);
"""

In [69]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [70]:
# create the tables with the schema files
with engine.connect() as connection:
    pass

### Add Data to Database

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [71]:
def write_dataframes_to_table(table_to_df_dict):
    for keys, values in table_to_df_dict.items():
        values.to_sql(name = keys, con=engine, if_exists="replace",index=False)
        

In [72]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_weather_data,
    "daily_weather": daily_weather_data,
}

In [73]:
write_dataframes_to_table(map_table_name_to_dataframe)

OperationalError: (sqlite3.OperationalError) database is locked
[SQL: PRAGMA main.table_info("taxi_trips")]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [64]:
!sqlite3 project.db < schema.sql

Error: near line 2: database is locked
Error: near line 10: database is locked
Error: near line 17: database is locked
Error: near line 32: database is locked


In [38]:
DATABASE_URL

'sqlite:///project.db'

In [47]:
a = pd.read_sql_table("taxi_trips",connection)
a

,pickup_datetime,dropoff_datetime,trip_distance,tip_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,Distance
0,2022-01-26 13:48:50,2022-01-26 13:51:33,0.57,1.00,-73.996971,40.742279,-73.996971,40.742279,0.000000
1,2022-01-13 23:53:57,2022-01-14 00:13:07,4.16,4.06,-73.965635,40.768615,-74.008984,40.735035,5.222630
2,2022-01-12 18:57:26,2022-01-12 19:08:02,1.18,1.92,-73.988787,40.753513,-73.996971,40.742279,1.426829
3,2022-01-04 07:44:15,2022-01-04 07:51:56,0.66,1.46,-73.968168,40.797962,-73.961764,40.809457,1.387213
4,2022-01-10 18:54:22,2022-01-10 19:06:22,3.00,1.00,-73.965146,40.756729,-74.001538,40.723888,4.768173
...,...,...,...,...,...,...,...,...,...
323259,2009-12-29 18:52:00,2009-12-29 19:00:00,1.82,0.00,-73.994457,40.755750,-74.002377,40.734517,2.453466
323260,2009-12-17 16:09:32,2009-12-17 16:33:40,6.10,0.00,-74.010487,40.718503,-73.982984,40.782506,7.484408
323261,2009-12-17 20:25:00,2009-12-17 20:30:00,0.88,0.00,-73.999313,40.730922,-73.987767,40.724363,1.215942
323262,2009-12-23 15:16:00,2009-12-23 15:21:00,1.19,0.00,-73.969417,40.756277,-73.957653,40.765547,1.429754


In [59]:
#connection = engine.connect()
#metadata = db.MetaData()
SQL_STMT = """
SELECT * 
FROM hourly_weather
"""
connection.execute(text(SQL_STMT)).fetchall()

[('2009-01-01T00:51:00', '0.0', 18.0),
 ('2009-01-01T01:51:00', '0.0', 18.0),
 ('2009-01-01T02:51:00', '0.0', 18.0),
 ('2009-01-01T03:51:00', '0.0', 8.0),
 ('2009-01-01T04:51:00', '0.0', 11.0),
 ('2009-01-01T05:51:00', '0.0', 18.0),
 ('2009-01-01T06:51:00', '0.0', 14.0),
 ('2009-01-01T07:51:00', '0.0', 8.0),
 ('2009-01-01T08:51:00', '0.0', 15.0),
 ('2009-01-01T09:51:00', '0.0', 8.0),
 ('2009-01-01T10:51:00', '0.0', 14.0),
 ('2009-01-01T11:51:00', '0.0', 14.0),
 ('2009-01-01T12:51:00', '0.0', 11.0),
 ('2009-01-01T13:51:00', '0.0', 14.0),
 ('2009-01-01T14:51:00', '0.0', 14.0),
 ('2009-01-01T15:51:00', '0.0', 5.0),
 ('2009-01-01T16:51:00', '0.0', 8.0),
 ('2009-01-01T17:51:00', '0.0', 5.0),
 ('2009-01-01T18:51:00', '0.0', 10.0),
 ('2009-01-01T19:51:00', '0.0', 11.0),
 ('2009-01-01T20:51:00', '0.0', 8.0),
 ('2009-01-01T21:51:00', '0.0', 7.0),
 ('2009-01-01T22:51:00', '0.0', 5.0),
 ('2009-01-01T23:51:00', '0.0', 3.0),
 ('2009-01-02T00:51:00', '0.0', 3.0),
 ('2009-01-02T01:51:00', '0.0', 0.0)

## Part 3: Understanding the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] For 01-2009 through 06-2015, what hour of the day was the most popular to take a yellow taxi? The result should have 24 bins.
* [ ] For the same time frame, what day of the week was the most popular to take an uber? The result should have 7 bins.
* [ ] What is the 95% percentile of distance traveled for all hired trips during July 2013?
* [ ] What were the top 10 days with the highest number of hired rides for 2009, and what was the average distance for each day?
* [ ] Which 10 days in 2014 were the windiest, and how many hired trips were made on those days?
* [ ] During Hurricane Sandy in NYC (Oct 29-30, 2012) and the week leading up to it, how many trips were taken each hour, and for each hour, how much precipitation did NYC receive and what was the sustained wind speed?

In [ ]:
def write_query_to_file(query, outfile):
    raise NotImplemented()

### Query N

_**TODO:** Write some prose that tells the reader what you're about to do here._

_Repeat for each query_

In [ ]:
QUERY_N = """
TODO
"""

In [ ]:
engine.execute(QUERY_N).fetchall()

In [ ]:
write_query_to_file(QUERY_N, "some_descriptive_name.sql")

## Part 4: Visualizing the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Create an appropriate visualization for the first query/question in part 3
* [ ] Create a visualization that shows the average distance traveled per month (regardless of year - so group by each month). Include the 90% confidence interval around the mean in the visualization
* [ ] Define three lat/long coordinate boxes around the three major New York airports: LGA, JFK, and EWR (you can use bboxfinder to help). Create a visualization that compares what day of the week was most popular for drop offs for each airport.
* [ ] Create a heatmap of all hired trips over a map of the area. Consider using KeplerGL or another library that helps generate geospatial visualizations.
* [ ] Create a scatter plot that compares tip amount versus distance.
* [ ] Create another scatter plot that compares tip amount versus precipitation amount.

_Be sure these cells are executed so that the visualizations are rendered when the notebook is submitted._

### Visualization N

_**TODO:** Write some prose that tells the reader what you're about to do here._

_Repeat for each visualization._

_The example below makes use of the `matplotlib` library. There are other libraries, including `pandas` built-in plotting library, kepler for geospatial data representation, `seaborn`, and others._

In [ ]:
# use a more descriptive name for your function
def plot_visual_n(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_n():
    # Query SQL database for the data needed.
    # You can put the data queried into a pandas dataframe, if you wish
    raise NotImplemented()

In [ ]:
some_dataframe = get_data_for_visual_n()
plot_visual_n(some_dataframe)